In [1]:
import pandas as pd
import numpy as np
import glob
home_dir = '/Users/zehaiwang/Downloads/2018NBA_Hackathon_Files/Business Analytics/'
file_list = glob.glob(home_dir+'*.csv')
print ([x.split('/')[-1].strip('.csv') for x in file_list])

['test_set', 'player_data', 'game_data', 'training_set']


In [13]:
test_df = pd.read_csv(file_list[0],parse_dates=["Game_Date"])
player_data = pd.read_csv(file_list[1],parse_dates=["Game_Date"])
game_data = pd.read_csv(file_list[2],parse_dates=["Game_Date"])
train_df = pd.read_csv(file_list[3],parse_dates=["Game_Date"])

In [28]:
# Sum viewers of all countries based on Game_ID
train_df = train_df.groupby(['Season','Game_ID','Game_Date','Away_Team','Home_Team'])['Rounded Viewers'].sum().reset_index()
train_df.rename(index=str, columns={"Rounded Viewers":"Total_Viewers"},inplace = True)
train_df.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600001,2016-10-25,NYK,CLE,57691
1,2016-17,21600002,2016-10-25,UTA,POR,40740
2,2016-17,21600003,2016-10-25,SAS,GSW,60428
3,2016-17,21600004,2016-10-26,MIA,ORL,22756
4,2016-17,21600005,2016-10-26,DAL,IND,33922


In [5]:
test_df.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600010,2016-10-26,DEN,NOP,NaN
1,2016-17,21600023,2016-10-28,HOU,DAL,NaN
2,2016-17,21600027,2016-10-29,BOS,CHA,NaN
3,2016-17,21600026,2016-10-29,ATL,PHI,NaN
4,2016-17,21600048,2016-11-01,LAL,IND,NaN


 #1. Create new features: Weekday, Month, VIP
 #2. Label encode categorical variables
 #3. Drop unused columns

In [29]:
full_df = [train_df, test_df]

In [30]:
# 1. Create new features: Weekday, Month, VIP
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays= cal.holidays()

for df in full_df:
    df['weekday'] = df.Game_Date.dt.weekday
    df['month'] = df.Game_Date.dt.month
    df['holiday']= df.Game_Date.isin(holidays)
    
# number ASG player in the Game
player_data['ASG_status'] = (player_data['ASG_Team']!= 'None')
ASG_per_Game = player_data.groupby('Game_ID')['ASG_status'].sum().to_frame().reset_index()

In [32]:
train_df = train_df.merge(ASG_per_Game, on='Game_ID')
test_df = test_df.merge(ASG_per_Game, on='Game_ID')

In [40]:
test_df[:10]

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers,weekday,month,holiday,ASG_status
0,2016-17,21600010,2016-10-26,7,18,NaN,2,10,False,1.0
1,2016-17,21600023,2016-10-28,10,6,NaN,4,10,False,1.0
2,2016-17,21600027,2016-10-29,2,3,NaN,5,10,False,2.0
3,2016-17,21600026,2016-10-29,0,22,NaN,5,10,False,1.0
4,2016-17,21600048,2016-11-01,13,11,NaN,1,11,False,1.0
5,2016-17,21600053,2016-11-01,28,26,NaN,1,11,False,2.0
6,2016-17,21600050,2016-11-01,19,8,NaN,1,11,False,1.0
7,2016-17,21600049,2016-11-01,25,15,NaN,1,11,False,1.0
8,2016-17,21600061,2016-11-02,4,2,NaN,2,11,False,2.0
9,2016-17,21600063,2016-11-02,24,23,NaN,2,11,False,0.0


### Label_encodeing

In [35]:
# Target and ID variables #
train_y = train_df["Total_Viewers"].values
test_id = test_df["Game_ID"].values

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics

# Label encode the categorical variables #
cat_vars = ["Away_Team","Home_Team"]
for col in cat_vars:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

In [41]:
cols_to_drop = ["Season","Game_ID","Game_Date","Total_Viewers"]
train_X = train_df.drop(cols_to_drop, axis=1)
test_X = test_df.drop(cols_to_drop, axis=1)

In [42]:
import lightgbm as lgb

OSError: dlopen(/usr/local/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/8/libgomp.1.dylib
  Referenced from: /usr/local/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found